In [4]:
import kwant
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import scipy.sparse.linalg as sla
from scipy.linalg import eig
from tqdm import tqdm
import pickle 
from pfapack import pfaffian as pf
from scipy.fft import ifftn
import dask.array as da
import scipy.integrate as integrate
from scipy.integrate import quad

In [5]:
#Define Hamiltonian, choose random k value, evaluate for A, G and test if PH operator sigmax cross sigmax flips its sign
def Na(a,g):
    return (4*np.exp(g/2)*(1+3*np.exp(2*g)))/((1+np.exp(g))**(4)*a)

def Nb(a,g):
    return (4*np.exp(3*g/2)*(3+np.exp(2*g)))/((1+np.exp(g))**(4)*a)


def H(k,A,G,eps1,eps2,d):
    

    N1=((1/np.cosh(G/2))**(4))*(-8*np.cosh(G/2)**(3)-4*np.sinh(G/2)**(2)*np.cos(3*k)+2*(3*np.cosh(G)+1)*np.cos(k))/(8*A) -eps1+d*np.cos(3*k)#checked
    N2=(-((1/np.cosh(G/2))**(4))/(4*A))*(4*np.cosh(G/2)**(3)-2*np.sinh(G/2)**(2)*np.cos(3*k)+3*np.cosh(G)*np.cos(k)+np.cos(k)) -eps2-d*np.cos(3*k) #checked
    Hm=-np.tanh(G/2)*((1/np.cosh(G/2))**(3)*(np.cosh(G)+4*1j*np.sinh(G/2)*np.sin(k)**(3)+2*np.cos(2*k)-1))/(2*A) -1j*d*np.sin(3*k)#checked
    S1=-0.5*np.cosh(G/2)*((1/np.cosh(G/2))**(4))*np.sin(k)*(-np.cosh(G)*np.cos(2*k)+np.cos(2*k)+2)  #checked
    S2=np.sqrt(2)*np.sqrt(np.cosh(G)+1)*np.tanh(G/2)*((1/np.cosh(G/2))**(3))*np.sin(k)*np.cos(k)*(1+1j*np.sinh(G/2)*np.sin(k))  #checked
    
    
    
    H = np.zeros((4,4), dtype=complex)
    H[0,0]=N1      
    H[0,1]=Hm      #+1j*d*np.sin(3*k)
    H[0,2]=S1
    H[0,3]=S2
    H[1,0]=Hm.conj()#    -1j*d*np.sin(3*k)
    H[1,1]=N2      
    H[1,2]=S2.conj()
    H[1,3]=-S1
    H[2,0]=S1
    H[2,1]=S2
    H[2,2]=-N1      #-eps1-d*np.cos(3*k)
    H[2,3]=-Hm      #-1j*d*np.sin(3*k)
    H[3,0]=S2.conj()
    H[3,1]=-S1
    H[3,2]=-Hm.conj()#          +1j*d*np.sin(3*k)
    H[3,3]=-N2        #   -eps2-d*np.cos(3*k)
    
    return(H)

def magical(k,A,G,eps1,eps2,d):
    #magic Unitary
    U = np.array([[1,1],[1j,-1j]], dtype='complex128')
    U = np.kron(U,np.eye(2))
    magic = (1/2)*(U@U)
    return (magic@H(k,A,G,eps1,eps2,d)@magic.conj().T)[:2, 2:]

id=np.array([[1,0],[0,1]])
sigma_x=np.array([[0,1],[1,0]])
sigma_z=np.array([[1,0],[0,-1]])



def complex_quadrature(func, a, b, **kwargs):
    def real_func(x):
        return np.real(func(x))
    def imag_func(x):
        return np.imag(func(x))
    real_integral = quad(real_func, a, b,limit=200)
    imag_integral = quad(imag_func, a, b,limit=200)
    return ((real_integral[0] + 1j*imag_integral[0]))

def integrand(k, a, g,eps1,eps2,d):
    k_res=0.001
    inv = (np.linalg.inv(magical(k,a,g,eps1,eps2,d)))
    der = (1/(2*k_res))*(magical(k+k_res,a,g,eps1,eps2,d)-magical(k-k_res,a,g,eps1,eps2,d))
    temp = inv@der
    return np.trace(temp)

In [6]:
k,A,G,eps1,eps2,d=1,1,1,0,0,0

In [21]:
C1 = np.kron(sigma_z,np.eye(2))
C2= np.kron(sigma_x,np.eye(2))
C3=C2@C1
eps,d=2,2
#verify commutation relations 
#time reversal
print('Time reversal C1 is',np.allclose(C1@H(-1,2,1,eps,eps,d).conj()@C1,H(1,2,1,eps,eps,d)))

#PH symmetry
print('PH C2 is',np.allclose(C2@np.conj(H(0.1,2,1,eps,eps,d))@C2+H(-0.1,2,1,eps,eps,d),0*np.eye(4)))

#weird chiral
print('Mysterious chiral C3=C2C1 is',np.allclose(C3@H(0.1,2,1,eps,eps,d)@C3,H(0.1,2,1,eps,eps,d)))

Time reversal C1 is True
PH C2 is True
Mysterious chiral C3=C2C1 is True


In [27]:
sigma_0 = np.array([[1, 0], [0, 1]])
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

array([[0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])

In [30]:
lat = kwant.lattice.chain(norbs=2)  #2 orbs per site given that we have p-h 
bulk = kwant.Builder(symmetry=kwant.lattice.TranslationalSymmetry([2]))

def onsite(site):
    return H(k,A,G,eps1,eps2,d)[:2,2:]

def hopping(site1, site2, t1):
    return H(k,A,G,eps1,eps2,d)[:2,:2]


bulk[lat(0)] = onsite
bulk[lat(1)] = onsite
bulk[lat(0),lat(1)] = hopping
bulk[lat(0),lat(2)] = hopping
bulk[lat(0),lat(3)] = hopping
bulk[lat(1),lat(3)] = hopping
bulk[lat(1),lat(4)] = hopping



finite = kwant.Builder()

L = 10

def shape(site):
    pos=site.pos
    return 0<=pos[0]<L

finite.fill(bulk, shape, start=[0]);

finite = finite.finalized()

kwant.plot(finite)

#######################

params_h1 = dict(k,A,G,eps1,eps2,d=1,1,1,0,0,0)
ham_1 = finite.hamiltonian_submatrix()


SyntaxError: positional argument follows keyword argument (3046716916.py, line 37)